In [1]:
import os
import sys
import numpy as np
from time import time

import keras
import tensorflow as tf

# Utilize CPU
# num_cores = 4
# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
#         inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
#         device_count = {'CPU' : 1, 'GPU' : 0})
# session = tf.Session(config=config)
# keras.backend.set_session(session)

from keras_pipeline.models import InceptionV3
from keras_pipeline.models import RetinaNet, RetinaNetConfig


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model_config = RetinaNetConfig(num_classes = 7, input_shape=(1333, 1333, 3))
model = RetinaNet(model_config)
model.output_shape

[(None, 300, 4), (None, 300), (None, 300)]

In [4]:
x = np.zeros((2,) + model.input_shape[1:])

t = time()
model.predict(x)
print(time() - t)


29.34926152229309


In [3]:
"""Get Model Anchor based on input_shape"""

def get_model_anchors():
    x = np.zeros((1,) + model.input_shape[1:])
    
    anchor_model = keras.models.Model(
        inputs  = model.input,
        outputs = model.get_layer('anchors').output
    )
    
    return anchor_model.predict(x)

from time import time
t = time()
model_anchors = get_model_anchors()
print(time() - t)
model_anchors.shape

4.788113117218018


(1, 320013, 4)

In [11]:
from keras_pipeline.utils import anchors

gen_anchors = anchors.compute_all_anchors((800, 800, 3), shapes_callback = model_config.compute_pyramid_feautre_shapes_for_img_shape)
gen_anchors.shape

(111798, 4)

2.939809117011683e-07

In [10]:
model_config.compute_anchors((800, 800))

array([[ -18.627417  ,   -7.3137085 ,   26.627417  ,   15.3137085 ],
       [ -24.50875898,  -10.25437949,   32.50875898,   18.25437949],
       [ -31.91878555,  -13.95939277,   39.91878555,   21.95939277],
       ...,
       [ 522.98066402,  341.96132803,  885.01933598, 1066.03867197],
       [ 475.92992816,  247.85985631,  932.07007184, 1160.14014369],
       [ 416.64971563,  129.29943127,  991.35028437, 1278.70056873]])

In [22]:
i = -4
model.layers[i].name, model.layers[i].output_shape

AttributeError: The layer "classification_submodel has multiple inbound nodes, with different output shapes. Hence the notion of "output shape" is ill-defined for the layer. Use `get_output_shape_at(node_index)` instead.